# Fine-tune BERT, RoBERTa & DistilBERT for Automatic Topic Classification

#### The code and notebook was adopted and modified for this task from the work of [Gabriele Catanese](https://github.com/cltl-students/catanese_gabriele_text_mining_thesis) who adopted and modified it from [George Mihaila](https://github.com/gmihaila)
#### Source code: https://gmihaila.medium.com/fine-tune-transformers-in-pytorch-using-transformers-57b40450635


## Info
This notebook is designed to use a pretrained BERT, RoBERTa, or DistilBERT model and fine-tune it on an Automatic Topic Classification task. All the instructions on the setups used can be found on "*Chapter 3: Methodology* " of the thesis report.

This notebook is using the [AutoClasses](https://huggingface.co/transformers/model_doc/auto.html) from 
[transformer](https://github.com/huggingface/transformers) by [Hugging Face](https://huggingface.co/) functionality. 
This functionality can guess a model's configuration, tokenizer and architecture just by passing in the model's name. This allows us to reuse the code on a large number of transformers models.

## How to use this notebook? 

This notebook was built with reusability in mind. The way I load the dataset into the PyTorch class is pretty standard and can be easily reused for any other dataset.

The only modifications needed to use your own dataset will be in reading in the dataset inside the PyTorch **Dataset** class under **Dataset and DataLoader** tab. The **DataLoader** will return a dictionary of batch inputs format so that it can be fed straight to the model using the statement: `outputs = model(**batch)`. *As long as this statement holds, the rest of the code will work.*

Basic parameters are defined under the **Imports** tab:
* `epochs` - will be used as the number of epochs to train the model. 
* `batch_size` - will be used as the batch size during training. The larger the batch the more RAM / GPU memory it will take. 
* `max_length` - I use this variable if I want to truncate text inputs to a shorter length than the maximum allowed word piece tokens sequence length. The shorter the sequence the faster it will train.
* `model_name_or_path` - This is where I put the transformer model I want to train. In this work, I used `bert-base-uncased`, `distilbert-base-uncased`, and `roberta-base`.
* `labels_ids` - It is mostly the case that labels have a name / meaning. We need to associate each name / meaning with a number / id. I use this variable to create a dictionary that maps labels names to ids. This will be used later inside the PyTorch Dataset class.



## Dataset
This notebook will cover the fine-tuning of transfromer-based models on Automatic Topic Classification. The Dataset used for this finetuning step is the annotated customer feedback set from tha Banking Domain. 

# Imports

Import all needed libraries for this notebook.


In [ ]:
import torch
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader       
from sklearn.metrics import classification_report, accuracy_score
from transformers import (AutoConfig, 
                          AutoModelForSequenceClassification, 
                          AutoTokenizer, AdamW, 
                          get_linear_schedule_with_warmup,
                          set_seed,
                          )
import pandas as pd
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn

Declare parameters used for this notebook:

In [ ]:
# Set seed for reproducibility,
set_seed(123)

# Number of training epochs
epochs = 4

# Number of batch_size - depending on the max sequence length and GPU memory.
batch_size = 32 

# Padd or truncate text sequences to a specific length
# if `None` it will use maximum sequence allowed by model.
# 128 is recommended length for batch size 32
max_length = 128

# Look for gpu to use. Will use `cpu` by default if no gpu found.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Name of transformers model - will use already pretrained model.
# Path of transformer model - will load your own model from local disk.
model_name_or_path = 'bert-base-uncased' #'bert-base-uncased' #'distilbert-base-uncased' #'roberta-base' 

dataversion = 'original' #'merged' #'original' #'other' #'backtranslation10' #'backtranslation20' #'undersampled'

# Dicitonary of labels and their id - this will be used to convert.
# String labels to number.
#these labels can be used for all datasets expect 'merged' and 'other' 
labels_ids = { 'Digital Options': 0, 'Employee Attitude and Behavior':1, 'Employee Contact':2, 'Employee Knowledge & Skills':3, 'Handling':4, 'Information Provision':5, 
 'No topic found':6, 'Overall Experience': 7, 'Price and Quality':8, 'Processes':9, 'Quality and Offering':10
                }    

#uncomment for merged labels
# labels_ids = { 'Handling' : 0, 'Employee' : 1, 'Digital Options': 2, 'Information Provision':3, 
#  'No topic found':4, 'Overall Experience': 5, 'Price and Quality':6, 'Processes':7
#                 }       

# #uncomment for 'other' labels
# labels_ids = {'Employee Attitude and Behavior' : 0, 'Employee Knowledge & Skills' : 1, 'Handling' : 2, 'Information Provision': 3, 
# 'No topic found':4, 'Other':5, 'Overall Experience': 6, 'Processes': 7}                                          


# How many labels are we using in training.
# This is used to decide size of classification head.
n_labels = len(labels_ids)

#path to the data
#training data
#original set as default
training_path = r'C:\Users\elena\Desktop\thesis\data\train.csv' #r'C:\Users\elena\Desktop\thesis\data\train_merged.csv' #r'C:\Users\elena\Desktop\thesis\data\train_other.csv'
                                                                        #r'C:\Users\elena\Desktop\thesis\data\train_backtranslation10.csv' #r'C:\Users\elena\Desktop\thesis\data\train_backtranslation20.csv'
                                                                        #r'C:\Users\elena\Desktop\thesis\data\train_undersampled.csv'
#validation
valid_path = r'C:\Users\elena\Desktop\thesis\data\valid.csv' #r'C:\Users\elena\Desktop\thesis\data\valid_merged.csv' #r'C:\Users\elena\Desktop\thesis\data\valid_other.csv'
                                                                     
#test 
test_path = r'C:\Users\elena\Desktop\thesis\data\test.csv' #r'C:\Users\elena\Desktop\thesis\data\test_merged.csv' #r'C:\Users\elena\Desktop\thesis\data\test_other.csv'

# Helper Functions

All Classes and functions that will be used in this notebook are kept under this section to help maintain a clean look of the notebook:

* CustomerFeedback
* train
* validation

In [ ]:
class CustomerFeedback(Dataset):
    r"""PyTorch Dataset class for loading data.

    This is where the data parsing happens and where the text gets encoded using
    loaded tokenizer.

    This class is built with reusability in mind: it can be used as is as long
    as the `dataloader` outputs a batch in dictionary format that can be passed 
    straight into the model - `model(**batch)`.

    Arguments:

    path (:obj:`str`):
        Path to the data partition.

    use_tokenizer (:obj:`transformers.tokenization_?`):
        Transformer type tokenizer used to process raw text into numbers.

    labels_ids (:obj:`dict`):
        Dictionary to encode any labels names into numbers. Keys map to 
        labels names and Values map to number associated to those labels.

    max_sequence_len (:obj:`int`, `optional`)
        Value to indicate the maximum desired sequence to truncate or pad text
        sequences. If no value is passed it will used maximum sequence size
        supported by the tokenizer and model.

    """

    def __init__(self, path, use_tokenizer, labels_ids, max_sequence_len=None):
        # Read in the data
        df_train = pd.read_csv(path, delimiter=';', header= 0, dtype= str, keep_default_na=False, encoding= 'utf-8')
        df_train = df_train.dropna()
        # Check max sequence length.
        max_sequence_len = 128
        # use_tokenizer.max_len if max_sequence_len is None else max_sequence_len
        texts = []
        labels = []
        print('Reading partitions...')
        # Since the labels are defined by folders with data we loop 
        # through each label.
        [labels.append(labels_ids[label]) for label in df_train['CsatTopicEn'].astype(str)] 
        [texts.append(sentence) for sentence in df_train['Sentence_new_improved'].astype(str)]   

        # Number of exmaples.
        self.n_examples = len(labels)
        # Use tokenizer on texts. This can take a while.
        print('Using tokenizer on all texts. This can take a while...')
        self.inputs = use_tokenizer(texts, add_special_tokens=True, truncation=True, padding=True, return_tensors='pt',  max_length=max_sequence_len)
        # Get maximum sequence length.
        self.sequence_len = self.inputs['input_ids'].shape[-1]
        print('Texts padded or truncated to %d length!' % self.sequence_len)
        # Add labels.
        self.inputs.update({'labels':torch.tensor(labels)})
        print('Finished!\n')

        return

    def __len__(self):
        r"""When used `len` return the number of examples.

        """

        return self.n_examples

    def __getitem__(self, item):
        r"""Given an index return an example from the position.

        Arguments:

          item (:obj:`int`):
              Index position to pick an example to return.

        Returns:
          :obj:`Dict[str, object]`: Dictionary of inputs that feed into the model.
          It holddes the statement `model(**Returned Dictionary)`.

        """

        return {key: self.inputs[key][item] for key in self.inputs.keys()}



def train(dataloader, optimizer_, scheduler_, device_):
    r"""
    Train pytorch model on a single pass through the data loader.

    It will use the global variable `model` which is the transformer model 
    loaded on `_device` that we want to train on.

    This function is built with reusability in mind: it can be used as is as long
    as the `dataloader` outputs a batch in dictionary format that can be passed 
    straight into the model - `model(**batch)`.

    Arguments:

      dataloader (:obj:`torch.utils.data.dataloader.DataLoader`):
          Parsed data into batches of tensors.

      optimizer_ (:obj:`transformers.optimization.AdamW`):
          Optimizer used for training.

      scheduler_ (:obj:`torch.optim.lr_scheduler.LambdaLR`):
          PyTorch scheduler.

      device_ (:obj:`torch.device`):
          Device used to load tensors before feeding to model.

    Returns:

      :obj:`List[List[int], List[int], float]`: List of [True Labels, Predicted
        Labels, Train Average Loss].
    """

    # Use global variable for model.
    global model

    # Tracking variables.
    predictions_labels = []
    true_labels = []
    # Total loss for this epoch.
    total_loss = 0

    # Put the model into training mode.
    model.train()

    # For each batch of training data...
    for batch in tqdm(dataloader, total=len(dataloader)):

        # Add original labels - use later for evaluation.
        true_labels += batch['labels'].numpy().flatten().tolist()

        # move batch to device
        batch = {k:v.type(torch.long).to(device_) for k,v in batch.items()}

        # Always clear any previously calculated gradients before performing a
        # backward pass.
        model.zero_grad()

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this a bert model function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(**batch)

        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple along with the logits. We will use logits
        # later to calculate training accuracy.
        loss, logits = outputs[:2]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()

        # Convert these logits to list of predicted labels values.
        predictions_labels += logits.argmax(axis=-1).flatten().tolist()

    # Calculate the average loss over the training data.
    avg_epoch_loss = total_loss / len(dataloader)
  
    # Return all true labels and prediction for future evaluations.
    return true_labels, predictions_labels, avg_epoch_loss



def validation(dataloader, device_):
    r"""Validation function to evaluate model performance on a 
    separate set of data.

    This function will return the true and predicted labels so we can use later
    to evaluate the model's performance.

    This function is built with reusability in mind: it can be used as is as long
    as the `dataloader` outputs a batch in dictionary format that can be passed 
    straight into the model - `model(**batch)`.

    Arguments:

    dataloader (:obj:`torch.utils.data.dataloader.DataLoader`):
          Parsed data into batches of tensors.

    device_ (:obj:`torch.device`):
          Device used to load tensors before feeding to model.

    Returns:

    :obj:`List[List[int], List[int], float]`: List of [True Labels, Predicted
        Labels, Train Average Loss]
    """

    # Use global variable for model.
    global model

    # Tracking variables
    predictions_labels = []
    true_labels = []
    #total loss for this epoch.
    total_loss = 0

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Evaluate data for one epoch
    for batch in tqdm(dataloader, total=len(dataloader)):

        # add original labels
        true_labels += batch['labels'].numpy().flatten().tolist()

        # move batch to device
        batch = {k:v.type(torch.long).to(device_) for k,v in batch.items()}

        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(**batch)

            # The call to `model` always returns a tuple, so we need to pull the 
            # loss value out of the tuple along with the logits. We will use logits
            # later to to calculate training accuracy.
            loss, logits = outputs[:2]

            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()

            # Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value 
            # from the tensor.
            total_loss += loss.item()

            # get predicitons to list
            predict_content = logits.argmax(axis=-1).flatten().tolist()

            # update list
            predictions_labels += predict_content

    # Calculate the average loss over the training data.
    avg_epoch_loss = total_loss / len(dataloader)

    # Return all true labels and prediciton for future evaluations.
    return true_labels, predictions_labels, avg_epoch_loss

# Load Model and Tokenizer

Loding the three esential parts of pretrained transformers: configuration, tokenizer and model. We also need to load model to the device we're planning to use (GPU / CPU).

In [ ]:
# Get model configuration.
print('Loading configuration...')
model_config = AutoConfig.from_pretrained(pretrained_model_name_or_path=model_name_or_path, 
                                          num_labels=n_labels)

# Get model's tokenizer.
print('Loading tokenizer...')
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name_or_path)

# Get the actual model.
print('Loading model...')
model = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name_or_path=model_name_or_path, 
                                                           config=model_config)

# Load model to defined device.
model.to(device)
print('Model loaded to `%s`'%device)

# Dataset and DataLoader

Create the PyTorch Dataset and DataLoader objects that will be used to feed data into our model.

In [ ]:
print('Dealing with Train...')
# Create pytorch dataset.
train_dataset = CustomerFeedback(path=training_path, 
                               use_tokenizer=tokenizer, 
                               labels_ids=labels_ids,
                               max_sequence_len=max_length)

print('Created `train_dataset` with %d examples!'%len(train_dataset))

# Move pytorch dataset into dataloader.
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
print('Created `train_dataloader` with %d batches!'%len(train_dataloader))

print()

print('Dealing with ...')
# Create pytorch dataset.
valid_dataset =  CustomerFeedback(path=valid_path, 
                               use_tokenizer=tokenizer, 
                               labels_ids=labels_ids,
                               max_sequence_len=max_length)

print('Created `valid_dataset` with %d examples!'%len(valid_dataset))

# Move pytorch dataset into dataloader.
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
print('Created `eval_dataloader` with %d batches!'%len(valid_dataloader))

# Train

Create optimizer and scheduler use by PyTorch in training.

Loop through the number of defined epochs and call the **train** and **validation** functions.

Outputs similar info after each epoch as in Keras: *train_loss:  - val_loss:  - train_acc: - valid_acc*.

After training, plot train and validation loss and accuracy curves to check how the training went.

In [ ]:
start_time = datetime.now()

# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
optimizer = torch.optim.AdamW(model.parameters(),
                  lr = 3e-5, # args.learning_rate 
                  eps = 1e-8 # args.adam_epsilon 
                  )

# Total number of training steps is number of batches * number of epochs.
# `train_dataloader` contains batched data so `len(train_dataloader)` gives 
# us the number of batches.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

# Store the average loss after each epoch so we can plot them.
all_loss = {'Epochs':[],'train_loss':[], 'val_loss':[]}
all_acc = {'Epochs':[], 'train_acc':[], 'val_acc':[]}

# Loop through each epoch.
print('Epoch')
for epoch in tqdm(range(epochs)):
    print()
    print('Training on batches...')
    # Perform one full pass over the training set.
    train_labels, train_predict, train_loss = train(train_dataloader, optimizer, scheduler, device)
    train_acc = accuracy_score(train_labels, train_predict)

    # Get prediction form model on validation data. 
    print('Validation on batches...')
    valid_labels, valid_predict, val_loss = validation(valid_dataloader, device)
    val_acc = accuracy_score(valid_labels, valid_predict)

    # Print loss and accuracy values to see how training evolves.
    print("  train_loss: %.5f - val_loss: %.5f - train_acc: %.5f - valid_acc: %.5f"%(train_loss, val_loss, train_acc, val_acc))
    print()

    # Store the loss value for plotting the learning curve.
    all_loss['train_loss'].append(train_loss)
    all_loss['val_loss'].append(val_loss)
    all_acc['train_acc'].append(train_acc)
    all_acc['val_acc'].append(val_acc)
    all_loss['Epochs'].append(epoch)
    all_acc['Epochs'].append(epoch)


end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

# Evaluate

When dealing with classification is useful to look at precision recall and F1 score.

A good gauge to have when evaluating a model is the confusion matrix.

In [ ]:
# SAVE MODELS TO THE MODELS FOLDER WITH TORCH
torch.save(tokenizer, r"C:\Users\elena\Desktop\thesis\models\r_aspect_tokenizer_3e")
torch.save(model, r"C:\Users\elena\Desktop\thesis\models\r_aspect_model_3e")
torch.save(device, r"C:\Users\elena\Desktop\thesis\models\r_aspect_device_3e")

In [ ]:
# # # load the model from disk
tokenizer = torch.load(r"C:\Users\elena\Desktop\thesis\models\r_aspect_tokenizer_3e")
model = torch.load(r"C:\Users\elena\Desktop\thesis\models\r_aspect_model_3e")
device = torch.load(r"C:\Users\elena\Desktop\thesis\models\r_aspect_device_3e")

In [ ]:
# Evaluation on test data.
# Create pytorch dataset.
test_dataset =  CustomerFeedback(path=test_path, 
                               use_tokenizer=tokenizer, 
                               labels_ids=labels_ids,
                               max_sequence_len=max_length)

print('Created `test_dataset` with %d examples!'%len(test_dataset))

# Move pytorch dataset into dataloader.
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
print('Created `test_dataloader` with %d batches!'%len(test_dataloader))

true_labels, predictions_labels, avg_epoch_loss = validation(test_dataloader, device)

# Create the evaluation report.
evaluation_report = classification_report(true_labels, predictions_labels, labels=list(labels_ids.values()), target_names=list(labels_ids.keys()), output_dict= True)
# Show the evaluation report.
print(f'Evalutation report for {model_name_or_path}, {dataversion}')
print(evaluation_report)
report = pd.DataFrame(evaluation_report).T
report = report.round(3)
report.to_csv(f'C:/Users/elena/Desktop/thesis/results/transformers/classification_report_{dataversion}_{model_name_or_path}.csv', sep = ';')


In [ ]:
fig, ax = plt.subplots(figsize = (12,8))
cm = sklearn.metrics.confusion_matrix(true_labels,predictions_labels)
ax.set_title(f'Confusion Matrix "{model_name_or_path}" - {dataversion} dataset')
matrix = sns.heatmap(cm, annot=True, xticklabels = list(labels_ids.keys()), yticklabels = list(labels_ids.keys()),  fmt='', cmap='Reds', ax = ax)
fig = matrix.get_figure()
outputfilepath = f"C:/Users/elena/Desktop/thesis/figures/transformers/confusion_matrix__{model_name_or_path}_{dataversion}.png"
plt.tight_layout()
fig.savefig(outputfilepath) 


In [ ]:
# Check examples of classification comparing gold label and prediction
# the output is hidden because it shows private data

test_df = pd.read_csv(test_path, delimiter=';', header= 0, dtype= str, keep_default_na=False, encoding= 'latin1')
sentences = test_df['Sentence_new_improved']

def get_key(val):
    """
    function to return the key 
    for any value in a dictionary
    """
    for key, value in labels_ids.items():
         if val == value:
            return key
 
    return "key doesn't exist"

i = 1
s_list = []
gold_list = []
pred_list = []

for s, gold, pred in zip(sentences, true_labels, predictions_labels):
    s_list.append(s)
    gold_list.append(get_key(gold))
    pred_list.append(get_key(pred))

    data = {'Sentence': s_list, 'Gold': gold_list, 'Prediction': pred_list}
    dataframe = pd.DataFrame(data)
    outputpath = f'C:/Users/elena/Desktop/thesis/results/predictions/transformers_{model_name_or_path}_{dataversion}.csv'
    dataframe.to_csv(outputpath, sep = ';', index = False)

Example output: 

SENTENCE 1 :   The website and app work great - every information I need is provided here. 
#
GOLD :  Digital Options     
PREDICTION: Information Provision

## End of the notebook